In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import numpy as np; np.set_printoptions(linewidth=110);
import pandas as pd
# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

# first we need a bit of import boilerplate
import os
import sys
from sys import platform
if platform == 'win32':
    sys.path.append('D:/Brain_Network/Code/')
    manifest_path = 'D:/Brain_Network/Data/Allen_Institute_Dataset/manifest.json'
    project_dir = 'D:/Brain_Network/'
elif platform =='darwin':
    sys.path.append('/Users/chenyu/Workspace/Brain_Network/Code/')
    manifest_path = '/Users/chenyu/Workspace/Brain_Network/Data/Allen_Institute_Dataset/manifest.json'
    project_dir = '/Users/chenyu/Workspace/Brain_Network/'
elif platform == 'linux':
    sys.path.append('/home/yuchen/workspace/Brain_Network/Code/')
    manifest_path = '/home/yuchen/workspace/Brain_Network/Data/Allen_Institute_Dataset/manifest.json'
    project_dir = '/home/yuchen/workspace/Brain_Network/'
else:
    print('Add new computer system settings.')

import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from tqdm.notebook import trange
import time
import itertools

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession

import data_visualizer
import data_model
import fine_time_coupling_model
import smoothing_spline
import util
import hierarchical_sampling_model
import hierarchical_model_generator
import jitter

# Data generator

In [2]:
generator = hierarchical_model_generator.HierarchicalModelGenerator()
data_dir = project_dir + 'Output/jitter/bivariate_coupling_data/'
trial_length = 5
trial_window = [0, trial_length]
num_trials = 200
beta = 30; beta_str = '30'  # ms
model_name = f'poisson_background_gaussian_mixture_square_full_model_beta{beta_str}ms_sigma100ms_trials{num_trials}_nonrepeated'

generator_par = {'num_trials': num_trials, 'trial_length': trial_length, 'trial_window': trial_window,
    'rho': 30, 'mu': 30, 'baseline': 10, 'window': 'gaussian', 'sigma': 0.1, 'random_seed': None,
    'type': 'square', 'alpha': [[0.5, -2.0], [2.0, 0.5]], 'beta': [[beta/1000, beta/1000], [beta/1000, beta/1000]], 'num_nodes': 2}
file_path = data_dir + model_name + '_generator_par.pkl'
util.save_variable(file_path, generator_par)

for itr in tqdm(range(75, 100), ncols=100, file=sys.stdout):
    generator_par['random_seed'] = itr
    spike_times = generator.generate_amarasingham_coupling_filter_spike_times_nonrepeated(generator_par, verbose=0)
    file_path = data_dir + model_name + f'_itr{itr}.pkl'
    util.save_variable(file_path, spike_times, verbose=False)

util.save_variable, save variable to:  D:/Brain_Network/Output/jitter/bivariate_coupling_data/poisson_background_gaussian_mixture_square_full_model_beta30ms_sigma100ms_trials200_nonrepeated_generator_par.pkl
100%|███████████████████████████████████████████████████████████████| 25/25 [13:06<00:00, 31.45s/it]


# Model fitting

In [5]:
jittertool = jitter.JitterTool()
data_dir = project_dir + 'Output/jitter/bivariate_coupling_data/'
model_dir = project_dir + 'Output/jitter/bivariate_coupling_model/poisson_background_gaussian_mixture_point_process_likelihood_regression/'

# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alphaN2_beta30ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha0_beta30ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_varying_sigma_80_140ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_sharerho20_nonsharerho10_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma20ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma5ms_trials200_nonrepeated'
# model_name = 'poisson_background_laplacian_mixture_square_alpha2_beta30ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_self_coupling_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_self_coupling15ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_source_self_coupling15ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_source_selfcouplingN15_10ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_target_selfcouplingN20_20ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_source_selfcouplingN20_20ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_target_selfcouplingN15_10ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_selfcoupling_sourceN15_10ms_targetN15_10ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_selfcoupling_sourceN15_10ms_targetN20_20ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_delays20ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_delaysN20ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_inhomorho_10_0_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_inhomorho_target0_source10_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_Source2Target_Target2Source_alpha2_beta30ms_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_Source2Target_alpha2_beta30ms_Target2Source_alpha01_sigma100ms_trials200_nonrepeated'
# model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma_100ms_delay10div64_trials200'
model_name = 'poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma12ms_trials200_nonrepeated'


file_path = data_dir + f'{model_name}_generator_par.pkl'
generator_par = util.load_variable(file_path, verbose=False)
trial_length = generator_par['trial_length']

num_itrs = 100
# kernel_widths = [5,20,30,80,100,130,160,200,250,500,800,'none']  # Fast search
# kernel_widths = [2,5,8,10,15,20,25,30,40,60,80,90,100,120,125,130,140,160,180,200,250,300,500,800,1000,'none']  # Full
# kernel_widths = [60,80,90,100,120,125,130,140,160,180,200,'none']  # narrow to best model.
kernel_widths = [180,200,250,300,]

for kernel_width in kernel_widths:
    model_par_list = []
    if kernel_width == 'none':
        model_par = {'filter_type': 'square', 'filter_length': generator_par['beta'][0][1],
                     'append_nuisance': ['const'],
                     'const_offset': 0, 'learning_rate': 0.5, 'max_num_itrs': 500, 'epsilon': 1e-5}
    else:
        model_par = {'filter_type': 'square', 'filter_length': generator_par['beta'][0][1],
                     'append_nuisance': ['const', 'gaussian_kernel'], 'kernel_width': kernel_width/1000,
                     'const_offset': 0, 'learning_rate': 0.5, 'max_num_itrs': 500, 'epsilon': 1e-5}

    trange = tqdm(range(num_itrs), ncols=100, file=sys.stdout)
    for itr in trange:
        file_path = data_dir + f'{model_name}_itr{itr}.pkl'
        spike_times = util.load_variable(file_path, verbose=False)
        spike_times_x, spike_times_y = spike_times[1], spike_times[0]
        model_par_hat = jittertool.bivariate_continuous_time_coupling_filter_regression(
                spike_times_x, spike_times_y, [0,trial_length], model_par)
        model_par_list.append(model_par_hat)

    file_path = model_dir + f'{model_name}_kernel{kernel_width}ms_model_par_list.pkl'
    util.save_variable(file_path, model_par_list)



100%|█████████████████████████████████████████████████████████████| 100/100 [12:54<00:00,  7.75s/it]
util.save_variable, save variable to:  D:/Brain_Network/Output/jitter/bivariate_coupling_model/poisson_background_gaussian_mixture_point_process_likelihood_regression/poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma12ms_trials200_nonrepeated_kernel180ms_model_par_list.pkl
100%|█████████████████████████████████████████████████████████████| 100/100 [12:39<00:00,  7.60s/it]
util.save_variable, save variable to:  D:/Brain_Network/Output/jitter/bivariate_coupling_model/poisson_background_gaussian_mixture_point_process_likelihood_regression/poisson_background_gaussian_mixture_square_alpha2_beta30ms_sigma12ms_trials200_nonrepeated_kernel200ms_model_par_list.pkl
100%|█████████████████████████████████████████████████████████████| 100/100 [12:28<00:00,  7.48s/it]
util.save_variable, save variable to:  D:/Brain_Network/Output/jitter/bivariate_coupling_model/poisson_background_gauss